In [ ]:
!pip install xgboost
!pip install seaborn 
!pip install matplotlib

In [ ]:
#### Imports para entrenamiento, predicción y evaluación del modelo ####
#### NO QUITAR ####
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
#### Código Agregado ####
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
### Loading Data
data = pd.read_csv('/home/bandrea/Documents/repos/DataScience/data/norech4w_20190908_abt_data.txt',sep='|')
tag = pd.read_csv('/home/bandrea/Documents/repos/DataScience/data/norech4w_20190908_abt_tag.txt',sep='|')

## Preparamos el tag y el dataset completo

In [ ]:
data.head()

In [ ]:
tag.head()

In [ ]:
'''
Observo que los campos SUSCRIBER_KEY tienen distinto formato en ambos data sets
Verifico si los 6 primeros digitos del campo SUSCRIBER_KEY en tag son todos iguales.
'''
tmp = tag['SUSCRIBER_KEY'].str[:6]
tmp.unique()

In [ ]:
#Como sí son todos iguales y no aportan información los elimino para que coincidan con los del dataset data
tag['SUSCRIBER_KEY'] = tag['SUSCRIBER_KEY'].str[6:]

In [ ]:
tag.head()

In [ ]:
'''
Por conocimiento de negocio se sabe que las columnas que contienen las letras AMT no son necesarias, 
asi que las borramos con un bucle for.
'''
tag.drop(columns=[x for x in tag.columns if 'AMT' in x],inplace=True)
tag.head()

In [ ]:
#creamos una nueva columna llamada total, que suma los  valores de las recargas acumuladas por semana. 
#Con axis=1 le decimos a sum() que la suma es por registro y no por columna
tag['Total'] = tag[tag.columns[1:]].sum(axis=1)
tag['Total']

In [ ]:
#entonces ahora borramos todas las columnas que contenga EVT, que son la cantidad de recargas acumuladas por semana, 
#al tener su valor acumulado ya estas columnas no nos sirven
tag.drop(columns=[x for x in tag.columns if 'EVT' in x],inplace=True)

In [ ]:
tag.head()

In [ ]:
#vamos a guardar en total, false para todos aquellos cuya sumatoria sea 0, y true para los que tengan un total mayor a 1 
tag['Total']=tag['Total']<1
tag['Total']

In [ ]:
#renombramos la columna total por target
tag.rename(columns={'Total':'Target'},inplace=True)
tag.head()

In [ ]:
#en vez de tenerlo como true o false es mas correcto manejarlo como 0s y 1s 
tag['Target'] = tag['Target'].astype(int)
tag.head()

In [ ]:
#y ahora vamos a mergear esta tabla con la tabla data 
# no habria ocurrencias de esta manera pq en data no modificamos el suscriber key entonces nunca se encontrarian dos iguales
#es por eso que agrego la celda de arriba con data['SUSCRIBER_KEY'] = data['SUSCRIBER_KEY'].str[6:]
fulldata = data.merge(tag, on='SUSCRIBER_KEY')
fulldata.head()

In [ ]:
#con este comando vemos cantidad de filas x columnas, ahora tenemos una columna mas en la tabla data, la columna target
fulldata.shape

In [ ]:
#tras haber hecho el merge de ambas tablas eliminamos los dataframes data y tag
del tag
del data


## Comprobemos columnas innecesarias

### Columnas que no aportan nada

In [ ]:
#veremos aquellas columnas que tienen el mismo valor en todas las filas (un unico valor, por eso el 1)
#entonces sabremos que estas columnas no aportan nada
[x for x in fulldata.columns if fulldata[x].nunique()==1]

### Por conocimiento de Dominio

In [ ]:
#por lo que ya nos dice el enunciado vamos a eliminar todas las columnas que no se usan según conocimiento del negocio
to_remove=["SPNDG_VOI_ONNET_ARPU_M1",
"SPNDG_VOI_ONNET_ARPU_M2",
"SPNDG_VOI_ONNET_ARPU_M3",
"SPNDG_VOI_OFFNET_ARPU_M1",
"SPNDG_VOI_OFFNET_ARPU_M2",
"SPNDG_VOI_OFFNET_ARPU_M3",
"USE_LCL_VOI_AMT_M1",
"USE_LCL_VOI_AMT_M2",
"USE_LCL_VOI_AMT_M3",
"SPNDG_VOI_INTRNTL_ARPU_M1",
"SPNDG_VOI_INTRNTL_ARPU_M2",
"SPNDG_VOI_INTRNTL_ARPU_M3",
"SMS_OFFNET_EXP_ARPU_AMT_M1",
"SMS_OFFNET_EXP_ARPU_AMT_M2",
"SMS_OFFNET_EXP_ARPU_AMT_M3",
"SMS_ONNET_EXP_ARPU_AMT_M1",
"SMS_ONNET_EXP_ARPU_AMT_M2",
"SMS_ONNET_EXP_ARPU_AMT_M3",
"EXPDTR_DATA_ARPU_AMT_M1",
"EXPDTR_DATA_ARPU_AMT_M2",
"EXPDTR_DATA_ARPU_AMT_M3"]


### Eliminamos las columnas

In [ ]:
#con esto eliminamos todas las columnas que eran innecesarias, y las 2 que encontramos que tenian el mismo valor
# para todas sus filas 
to_remove = to_remove + [x for x in fulldata.columns if fulldata[x].nunique()==1]

In [ ]:
to_remove

In [ ]:
fulldata.drop(columns=to_remove,inplace=True)

In [ ]:
#Eliminamos las columnas PREP_RECH_Q_EVT dejando la de granularidad mas chica--ANDREA
to_remove_PREP_RECH_Q_EV=['PREP_RECH_Q_EVT_X1',
                          'PREP_RECH_Q_EVT_X2',
                          'PREP_RECH_Q_EVT_X3',
                          'PREP_RECH_Q_MON_12W',
                          'PREP_RECH_Q_TUE_12W',
                          'PREP_RECH_Q_WEN_12W',
                          'PREP_RECH_Q_THUR_12W',
                          'PREP_RECH_Q_FRI_12W',
                          'PREP_RECH_Q_SAT_12W',
                          'PREP_RECH_Q_SUN_12W',
                         ]
fulldata.drop(columns=to_remove_PREP_RECH_Q_EV,inplace=True)

In [ ]:
#de 213 columnas pasamos a 190, good! 
fulldata.shape

### Chequeamos duplicados para todas las columnas

In [ ]:
#vemos si hay duplicados
fulldata.duplicated().any()

In [ ]:
#chequeamos la cantidad de filas antes de aplicar drop_duplicates
fulldata.shape

In [ ]:
#nos aseguramos que la cantidad de registros efectivamente es el mismo
fulldata.drop_duplicates().shape

## Ahora Chequeemos faltantes

In [ ]:
#haremos un heatmap con las primeras 15 columnas identificando donde hay nulos 
#donde vemos mas blancos es que hay mas nulos
sns.heatmap(fulldata[fulldata.columns[0:15]].isnull(), cbar=False)
#vemos asi que network_tech tiene muuuchos nulos 

In [ ]:
#ahora vamos a hacer lo mismo pero con las primeras 50 columnas
sns.heatmap(fulldata[fulldata.columns[0:50]].isnull(), cbar=False)

In [ ]:
#tomarwemos una muestra de fulldata de un 30%
sampledata = fulldata.sample(frac=0.3)

In [ ]:
#y de esta muestra haremos un heatmap para poder visualizar mejor 
sns.heatmap(sampledata.isnull(), cbar=False)

In [ ]:
#contaremos la cantidad de nulos para cada columna 
nullcount = {col:fulldata[col].isnull().sum() for col in fulldata.columns}

In [ ]:
#y haremos una serie con estos valores para graficarlo como grafico de barras
missing = pd.Series(nullcount)

In [ ]:
#ahora graficamos valores relativos solamente de las variables que contienen nulos, por eso usamos div
missing[missing>0].div(fulldata.shape[0]).plot.bar()

#### Imputar Network Tech

In [ ]:
#como hay muuuchos nulos en network_tech vamos a trabajar con esto 
#sacaremos un promedio de sus valores y vemos que la tecnologia con mas ocurrencias es LTE
fulldata['NETWORK_TECH'].value_counts(normalize=True,dropna=False)

In [ ]:
#entonces para cada valor nulo existente en network_tech, le asignaremos el valor LTE, esta es una manera de eliminar nulos
fulldata.loc[fulldata['NETWORK_TECH'].isnull(),'NETWORK_TECH']='LTE'

In [ ]:
#obtenemos la nueva distribucion luego de asignar LTE
fulldata['NETWORK_TECH'].value_counts(normalize=True)

In [ ]:
#ahora trabajaremos con las columnas DEVICE_MODEL_NAME y DEVICE_VENDOR_NAME que tambien poseen muchos nulos
#device model name - distrucion por modelo
fulldata['DEVICE_MODEL_NAME'].value_counts(normalize=True,dropna=False)

In [ ]:
fulldata['DEVICE_VENDOR_NAME'].value_counts(normalize=True,dropna=False)

In [ ]:
'''
observamos que en ambas variables se utiliza el valor NOT_IDENTIFIED para los registros 
que no poseen datos del device, entonces en este caso, a los valores nulos los 
reemplazaremos por NOT_IDENTIFIED.
'''
fulldata.loc[fulldata['DEVICE_MODEL_NAME'].isnull(),'DEVICE_MODEL_NAME']='NOT_IDENTIFIED'
fulldata.loc[fulldata['DEVICE_VENDOR_NAME'].isnull(),'DEVICE_VENDOR_NAME']='NOT_IDENTIFIED'

In [ ]:
# Asigno en una categoria Others los que no llega a los que los vendors tiene una representacion menor que 100 
# creo la nueva categoria Others 
# contar cuantos hay y los que no llegan a 100 los reemplazo por la categoria Other
vendors = fulldata['DEVICE_VENDOR_NAME'].value_counts()
vendors_to_replace = vendors[vendors<100].index
fulldata.loc[fulldata['DEVICE_VENDOR_NAME'].isin(vendors_to_replace),'DEVICE_VENDOR_NAME'] ='OTHER'

In [ ]:
#hago lo mismo con la columna DEVICE_MODEL_NAME
# contar cuantos hay y los que no llegan a 100 los reemplazo por la categoria Other
models = fulldata['DEVICE_MODEL_NAME'].value_counts()
models_to_replace = models[models<100].index
fulldata.loc[fulldata['DEVICE_MODEL_NAME'].isin(models_to_replace),'DEVICE_MODEL_NAME'] ='OTHER'

In [ ]:
#las otras 4 columnas que contienen valores nulos son LAT_PROV_BTS , LON_PROV_BTS, LAT_CITY_BTS y LON_CITY_BTS
fulldata[fulldata.columns[:5]]

In [ ]:
fulldata['LAT_PROV_BTS'].value_counts(normalize=True,dropna=False)

In [ ]:
fulldata['LON_PROV_BTS'].value_counts(normalize=True,dropna=False)

In [ ]:
#puedo observar que los mismos registros que no proveen informacion sobre latitutd de la provincia
#tampoco lo proveen sobre su longitud. y como no es un numero muy significativo puedo proceder a eliminarlos

#haremos el mismo analisis para ver los nulos de latitud y longitud de ciudad
fulldata['LAT_CITY_BTS'].value_counts(normalize=True,dropna=False)

In [ ]:
fulldata['LON_CITY_BTS'].value_counts(normalize=True,dropna=False)

In [ ]:
#no vemos en las salidas anteriores la cantidad de nulos, entonces llamaremos al metodo isnull y veremos 
#una sumatoria para ver si la cantidad de nulos es igual
fulldata['LAT_CITY_BTS'].isnull().sum()

In [ ]:
fulldata['LON_CITY_BTS'].isnull().sum()

In [ ]:
fulldata['LAT_PROV_BTS'].isnull().sum()

In [ ]:
fulldata['LON_PROV_BTS'].isnull().sum()

In [ ]:
#tras ver que es exactamente la misma cantidad de registros los que poseen valores nulos, vamos a proceder a eliminar los 
#mismos ya que consideramos que no nos modificaran considerablemente la muestra 
fulldata.drop(fulldata[fulldata['LAT_PROV_BTS'].isnull()].index, inplace = True)

In [ ]:
#habiendo eliminado todos los registros, veremos que ya no poseemos mas valores nulos en ninguna de nuestras columnas
{col:fulldata[col].isnull().sum() for col in fulldata.columns}

In [ ]:
fulldata.shape

In [ ]:
#aca se fija cuantos registros hay tras hacer drop NaN hay para corroborar que ya no quedan
fulldata.dropna().shape

### Manejo de Irregulares y Outliers

In [ ]:
#obtengo todas las columnas
fulldata.columns.to_list()

#### Descripción Estadística

In [ ]:
#trd_m1 es el trafico de datos mensual
fulldata['TRD_M1'].describe()

In [ ]:
fulldata['TRD_M1'].plot.hist(bins=100)

In [ ]:
#hacemos un boxplot con los traficos de datos mensuales para identificar outliers
fulldata.boxplot(['TRD_M1','TRD_M2','TRD_M3'])

In [ ]:
#ahora haremos un boxplot y un violinplot solo del trafico de datos mensual del mes 1 
fulldata.boxplot('TRD_M1')

In [ ]:
sns.violinplot(y=fulldata['TRD_M1'])

In [ ]:
fulldata['SEGMENTATION'].value_counts().plot.bar()

In [ ]:
fulldata['DEVICE_VENDOR_NAME'].value_counts()[:20].plot.bar()

### TRATAMIENTO DE OUTLIERS

In [ ]:
fulldata.shape

In [ ]:
#Eliminando valores muy extremos
fulldata[fulldata['TRD_M1']<=fulldata['TRD_M1'].quantile(0.99)].boxplot('TRD_M1')


In [ ]:
#Eliminando valores muy extremos -toma el 99% DE LOS VALORES 
fulldata[fulldata['TRD_M1']<=fulldata['TRD_M1'].quantile(0.99)].shape

In [ ]:
fulldata[fulldata['TRD_M1']>fulldata['TRD_M1'].quantile(0.95)].shape

In [ ]:
#Aqui reemplazomos los outliers por el valor mas alto del quantile 0.95, y de esta forma ya no tenemos outliers
fulldata.loc[fulldata['TRD_M1']>fulldata['TRD_M1'].quantile(0.95),'TRD_M1'] = fulldata['TRD_M1'].quantile(0.95)

In [ ]:
fulldata.boxplot('TRD_M1')

Hacemos el mismo tratamiento de outliers para los traficos de datos mensual 2 y 3

In [ ]:
#Antes de eliminar outliers
fulldata.boxplot('TRD_M2')

In [ ]:
#Observamos cuantos registros hay con valores por debajo del maximo valor del quantile 0.99
fulldata[fulldata['TRD_M2']<=fulldata['TRD_M2'].quantile(0.99)].shape

In [ ]:
#Observamos cuantos registros hay con valores por debajo del maximo valor del quantile 0.95
fulldata[fulldata['TRD_M2']>fulldata['TRD_M2'].quantile(0.95)].shape

In [ ]:
fulldata[fulldata['TRD_M2']<=fulldata['TRD_M2'].quantile(0.99)].boxplot('TRD_M2')

In [ ]:
#Asignamos a los outliers el valor mas alto del quantile 0.95
fulldata.loc[fulldata['TRD_M2']>fulldata['TRD_M2'].quantile(0.95),'TRD_M2'] = fulldata['TRD_M2'].quantile(0.95)

In [ ]:
#despues de eliminar outliers
fulldata.boxplot('TRD_M2')

In [ ]:
#Observamos las estadísticas de TRD_M3
fulldata['TRD_M3'].describe()

In [ ]:
#antes de eliminar outliers
fulldata.boxplot('TRD_M3')

In [ ]:
#Observamos cuantos registros hay con valores por debajo del maximo valor del quantile 0.99
fulldata[fulldata['TRD_M3']<=fulldata['TRD_M3'].quantile(0.99)].shape

In [ ]:
#Observamos cuantos registros hay con valores por debajo del maximo valor del quantile 0.95
fulldata[fulldata['TRD_M3']>fulldata['TRD_M3'].quantile(0.95)].shape

In [ ]:
#Asignamos a los outliers el valor mas alto del quantile 0.95
fulldata.loc[fulldata['TRD_M3']>fulldata['TRD_M3'].quantile(0.95),'TRD_M3'] = fulldata['TRD_M3'].quantile(0.95)

In [ ]:
#despues de eliminar outliers
fulldata.boxplot('TRD_M3')

In [ ]:
#podemos ver los traficos de datos mensuales de los 3 meses ahora sin outliers 
fulldata.boxplot(['TRD_M1','TRD_M2','TRD_M3'])

Ahora haremos el mismo tratamiento de outliers para el trafico de datos en streaming en los meses 1, 2 y 3

In [ ]:
fulldata['TRD_STR_M1'].describe()

In [ ]:
fulldata['TRD_STR_M1'].plot.hist()

In [ ]:
sns.violinplot(fulldata['TRD_STR_M1'])

In [ ]:
#Graficamos suprimiendo valores muy extremos -toma el 99% DE LOS VALORES 
fulldata[fulldata['TRD_STR_M1']<=fulldata['TRD_STR_M1'].quantile(0.99)].boxplot('TRD_STR_M1')

In [ ]:
fulldata[fulldata['TRD_STR_M1']<=fulldata['TRD_STR_M1'].quantile(0.99)].shape

In [ ]:
fulldata[fulldata['TRD_STR_M1']>fulldata['TRD_STR_M1'].quantile(0.95)].shape

In [ ]:
#Aquí reemplazamos los outliers con el valor del quantile 0.95
fulldata.loc[fulldata['TRD_STR_M1']>fulldata['TRD_STR_M1'].quantile(0.95),'TRD_STR_M1'] = fulldata['TRD_STR_M1'].quantile(0.95)

In [ ]:
fulldata.boxplot('TRD_STR_M1')

Hacemos el mismo tratamiento de outliers para TRAFICO DE DATOS MENSAJES POR INTERENT MENSUAL 1, 2 y 3

In [ ]:
#Observamos las estadísticas de 'TRD_SN_M1'
fulldata['TRD_SN_M1'].describe()

In [ ]:
#Vemos el boxplot para identificar outliers
fulldata.boxplot('TRD_SN_M1')

In [ ]:
fulldata[fulldata['TRD_SN_M1']<=fulldata['TRD_SN_M1'].quantile(0.99)].shape

In [ ]:
fulldata[fulldata['TRD_SN_M1']>fulldata['TRD_SN_M1'].quantile(0.95)].shape

In [ ]:
#Graficamos suprimiendo valores muy extremos -toma el 95% DE LOS VALORES 
fulldata[fulldata['TRD_SN_M1']<=fulldata['TRD_SN_M1'].quantile(0.95)].boxplot('TRD_SN_M1')

Observamos que suprimiendo los valores del quantile 0.95 estamos modificando mucho la media del dataset

In [ ]:
#Procedemos a reemplazar outliers con el valor del quantile 0.99
fulldata.loc[fulldata['TRD_SN_M1']>fulldata['TRD_SN_M1'].quantile(0.99),'TRD_SN_M1'] = fulldata['TRD_SN_M1'].quantile(0.99)

In [ ]:
#distribución tras eliminar outliers, manteniendo sin grandes cambios los datos estadísticos del data frame
fulldata.boxplot('TRD_SN_M1')

In [ ]:
#Observamos las estadísticas de 'TRD_SN_M2'
fulldata['TRD_SN_M2'].describe()

In [ ]:
#Vemos el boxplot para identificar outliers
fulldata.boxplot('TRD_SN_M2')

In [ ]:
#Graficamos suprimiendo valores muy extremos -toma el 95% DE LOS VALORES 
fulldata[fulldata['TRD_SN_M2']<=fulldata['TRD_SN_M2'].quantile(0.95)].boxplot('TRD_SN_M2')

Observamos que suprimiendo los valores del quantile 0.95 estamos modificando mucho la media del dataset

In [ ]:
#Procedemos a reemplazar outliers con el valor del quantile 0.99
fulldata.loc[fulldata['TRD_SN_M2']>fulldata['TRD_SN_M2'].quantile(0.99),'TRD_SN_M2'] = fulldata['TRD_SN_M2'].quantile(0.99)

In [ ]:
#distribución tras eliminar outliers, manteniendo sin grandes cambios los datos estadísticos del data frame
fulldata.boxplot('TRD_SN_M2')

In [ ]:
#Observamos las estadísticas de 'TRD_SN_M3'
fulldata['TRD_SN_M3'].describe()

In [ ]:
#Vemos el boxplot para identificar outliers
fulldata.boxplot('TRD_SN_M3')

In [ ]:
#Graficamos suprimiendo valores muy extremos -toma el 95% DE LOS VALORES 
fulldata[fulldata['TRD_SN_M3']<=fulldata['TRD_SN_M3'].quantile(0.95)].boxplot('TRD_SN_M3')

Observamos que suprimiendo los valores del quantile 0.95 estamos modificando mucho la media del dataset

In [ ]:
#Procedemos a reemplazar outliers con el valor del quantile 0.99
fulldata.loc[fulldata['TRD_SN_M3']>fulldata['TRD_SN_M3'].quantile(0.99),'TRD_SN_M3'] = fulldata['TRD_SN_M3'].quantile(0.99)

In [ ]:
#distribución tras eliminar outliers, manteniendo sin grandes cambios los datos estadísticos del data frame
fulldata.boxplot('TRD_SN_M3')

Ahora vamos a hacer lo mismo pero con el TRAFICO DE DATOS MENSAJES POR INTERENT MENSUAL

In [ ]:
sns.boxplot(fulldata['TRD_IM_M1'])

In [ ]:
fulldata['TRD_IM_M1'].describe()

In [ ]:
#Eliminando valores muy extremos -toma el 99% DE LOS VALORES 
fulldata[fulldata['TRD_IM_M1']<=fulldata['TRD_IM_M1'].quantile(0.95)].boxplot('TRD_IM_M1')

In [ ]:
fulldata[fulldata['TRD_IM_M1']<=fulldata['TRD_IM_M1'].quantile(0.99)].shape

In [ ]:
fulldata[fulldata['TRD_IM_M1']>fulldata['TRD_IM_M1'].quantile(0.95)].shape

In [ ]:
fulldata.loc[fulldata['TRD_IM_M1']>fulldata['TRD_IM_M1'].quantile(0.95),'TRD_IM_M1'] = fulldata['TRD_IM_M1'].quantile(0.95)

In [ ]:
fulldata.boxplot('TRD_IM_M1')

Ahora vamos a hacer lo mismo con el MONTO DE RECARGAS ACUMULADAS MENSUALES--ANDRE

In [ ]:
#vamos a comparara los montos de recargas acumuladas Mensuales - desp comparo cada una contra el target --ANDREA
fulldata.boxplot(['PREP_RECH_AMT_X1','PREP_RECH_AMT_X2','PREP_RECH_AMT_X3'])

Ahora vamos a hacer lo mismo con el TRAFICO DE DATOS DE OTRAS FUENTES MENSUALES-- ANDRE

In [ ]:
#vamos a comparar trafico de datos de otras fuentes mensual --ANDREA
fulldata.boxplot(['TRD_OTH_M1','TRD_OTH_M2','TRD_OTH_M3'])
#Los borramos porque estan inconsistennnteeees

Eliminas las columnas TRAFICO DE DATOS DE OTRAS FUENTES MENSUALES porque como vimos en el 
grafico anterior son inconsistentes-- ANDRE

In [ ]:
#Eliminamos columnas con datos incorrectos -- VER GRAFICO ANDRE 
column_incorrect = ['PREP_RECH_NDAYS_LASTRECH_12W',
                   'TRD_OTH_M1',
                   'TRD_OTH_M2',
                   'TRD_OTH_M3']
fulldata.drop(columns=column_incorrect,inplace=True)#--ANDREA

In [ ]:
fulldata['TRS_ONNET_SMS_M1'].describe()

In [ ]:
fulldata[fulldata.columns[:20]]

In [ ]:
#vemos los datos unicos que contienen las primeras 20 columnas
fulldata['VALUE_SEGMENT'].unique()

In [ ]:
fulldata['SEGMENTATION'].unique()

In [ ]:
fulldata['MICROSEGMENTATION'].unique()

In [ ]:
fulldata['DEVICE_VENDOR_NAME'].unique()

### Transformación de Datos

In [ ]:
#observa el tipo de dato de activation date y luego lo convierte a datetime- es tipo objeto
fulldata['COMMERCIAL_ACTIVATION_DATE'].dtype

In [ ]:
fulldata['COMMERCIAL_ACTIVATION_DATE']
#podemos ver que se representan las fechas como string 

In [ ]:
#entonces realizaremos una transformacion para tratarlas como data 
fulldata['COMMERCIAL_ACTIVATION_DATE'] = pd.to_datetime(fulldata['COMMERCIAL_ACTIVATION_DATE'])

In [ ]:
#cuando consultamos los valores de value_segment, vemos que los clientes se dividen en 4 categorias
#sera mas facil trabajar con la categoria considerandolo como un valor numerico, sin importa como se llame dicha categoria
fulldata['VALUE_SEGMENT'].unique()

In [ ]:
fulldata['VALUE_SEGMENT'] = pd.to_numeric(fulldata['VALUE_SEGMENT'].str.split(' - ').str[0])

In [ ]:
#podemos realizar el mismo trabajo para el caso de la columna SEGMENTATION, nos quedamos con los valores numericos 
#y no con las siglas de cada una de ellas 
fulldata['SEGMENTATION'].unique()

In [ ]:
fulldata['SEGMENTATION'] = pd.to_numeric(fulldata['SEGMENTATION'].str.split('-').str[0])

In [ ]:
#vemos como nos queda nuestro data set luego de las respectivas transformaciones realizadas
fulldata.head()

## Distribución del Dataset

### Univariadas

In [ ]:
#veremos en un grafico la cantidad de recargas acumuladas mensuales en el mes 1 --ANDRE TUVE QUE CAMBIARA ACA PORQUE BORRE EL MES
sns.distplot(fulldata['PREP_RECH_Q_EVT_W1'])

In [ ]:
sns.kdeplot(fulldata['PREP_RECH_Q_EVT_W1'],shade=True)

In [ ]:
#comparo el target de los que se van con el value segment 
fulldata[fulldata['Target']==1]['VALUE_SEGMENT'].value_counts().plot.bar()

In [ ]:
#sumamos los full data y luego borramos las columnas de PACK_DATA_EXP_Wi
fulldata['TOTAL_EXP_PACKDATA_W'] = fulldata[fulldata.columns[178:188]].sum(axis=1)
#elimino lo que ya sume
fulldata.drop(columns=[x for x in fulldata.columns if 'PACK_DATA_EXP_W' in x],inplace=True)
fulldata['TOTAL_EXP_PACKDATA_W']=fulldata['TOTAL_EXP_PACKDATA_W']<1
fulldata['TOTAL_EXP_PACKDATA_W'] = fulldata['TOTAL_EXP_PACKDATA_W'].astype(int)

In [ ]:
fulldata.columns.to_list()

In [ ]:
#grafico donde relacionamos la cantidad de pack de datos mensuales con el trafico de datos de mensaje por internet 
#mensual, en el mes 1
sns.jointplot(x=fulldata['PACK_DATA_Q_X1'], y=fulldata['TRD_IM_M1'])

In [ ]:
#TRD_M1 = trafico de datos mensual
#TRD_STR_M1 = trafico de datos en streaming mensual
#TRD_SN_M1 = trafico de datos en redes sociales mensual
#TRD_IM_M1 = trafico de datos de mensajes por internet mensual

sns.pairplot(fulldata[['TRD_M1','TRD_STR_M1','TRD_SN_M1','TRD_IM_M1']])

In [ ]:
#ANDREA - Analizamos la variable TERNURE_CUSTOMER
fulldata.boxplot(['TENURE_CUSTOMER']) 
fulldata.shape

In [ ]:
#Vemos como queda el grafico si eliminariamos valores muy extremos
fulldata[fulldata['TENURE_CUSTOMER']<=fulldata['TENURE_CUSTOMER'].quantile(0.99)].boxplot('TENURE_CUSTOMER')
fulldata.shape

In [ ]:
fulldata.loc[fulldata['TENURE_CUSTOMER']>fulldata['TENURE_CUSTOMER'].quantile(0.99)]=fulldata['TENURE_CUSTOMER'].quantile(0.99)

In [ ]:
#ANDRE LAST
fulldata.boxplot(['TENURE_CUSTOMER'])
fulldata.shape

In [ ]:

#Comparo Antiguedad contra el target 0 y 1 --ANDRE
Tenure_1 = fulldata[fulldata["Target"] == 1]["TENURE_CUSTOMER"].value_counts(normalize=True).sort_index()
Tenure_0 = fulldata[fulldata["Target"] == 0]["TENURE_CUSTOMER"].value_counts(normalize=True).sort_index()
#fulldata[fulldata['SUSCRIBER_KEY']=='E_68006527_20110525']["TENURE_CUSTOMER"] 

In [ ]:
# --ANDRE

fig = go.Figure(data=[
    go.Bar(name='Target1- Se va',y=Tenure_1.values,x=Tenure_1.index),
    go.Bar(name='Target0- Se queda',y=Tenure_0.values,x=Tenure_0.index)
])
fig.update_layout(xaxis=dict(range=[0,60]))
fig.update_layout(barmode='group')

fig.update_layout(
    title="Comparasion Targets con Tenure Customer",
    xaxis_title="Antiguedad cliente meses",
    yaxis_title="Fr. Relativa",

)

fig.show()

In [ ]:
#ANDREA - Analizamos la variable TERNURE_CUSTOMER_BL- Antiguedad desde el primer gasto
fulldata.boxplot(['TENURE_CUSTOMER_BL']) 
fulldata.shape

In [ ]:

#Comparo Antiguedad contra el target 0 y 1 --ANDRE
TenureBL_1 = fulldata[fulldata["Target"] == 1]["TENURE_CUSTOMER_BL"].value_counts(normalize=True).sort_index()
TenureBL_0 = fulldata[fulldata["Target"] == 0]["TENURE_CUSTOMER_BL"].value_counts(normalize=True).sort_index()


In [ ]:
#ind=np.arange(fulldata['TENURE_CUSTOMER'].nunique()) 
#width = 0.40  # ancho de las barras

fig = go.Figure(data=[
    go.Bar(name='Target1- Se va',y=TenureBL_1.values,x=TenureBL_1.index),
    go.Bar(name='Target0- Se queda',y=TenureBL_0.values,x=TenureBL_0.index)
])
# Change the bar mode
fig.update_layout(xaxis=dict(range=[0,60]))
fig.update_layout(barmode='group')


fig.update_layout(
    title="Comparasion Targets con Antiguedad desde que hizo el primer gasto",
    xaxis_title="Antiguedad desde que hizo el primer gasto",
    yaxis_title="Fr. Relativa",

)

fig.show()

In [ ]:
#Comparasion entre  TENURE_CUSTOMER y  TENURE_CUSTOMER_BL SON IGUALES LA ELIMINO A TERNURE_COSTUMER_BL

for_pairplot= fulldata[["TENURE_CUSTOMER", "TENURE_CUSTOMER_BL"]]
for_pairplot=for_pairplot.sample(1000)
sns.pairplot(for_pairplot)

In [ ]:
#Borro la columna TENURE_CUSTOMER_BL
fulldata.drop(columns="TENURE_CUSTOMER_BL", inplace=True)

Comparacion entre Montos pack voz mensual y Cantidad de pack voz mensual  PACK_VOICE_Q_X1 PACK_VOICE_Q_X2 PACK_VOICE_Q_X3 PACK_VOICE_AMT_X1 PACK_VOICE_AMT_X2 PACK_VOICE_AMT_X3 --ANDRE







In [ ]:
#Comparacion entre Montos pack voz mensual y Cantidad de pack voz mensual  PACK_VOICE_Q_X1 PACK_VOICE_Q_X2 PACK_VOICE_Q_X3 PACK_VOICE_AMT_X1 PACK_VOICE_AMT_X2 PACK_VOICE_AMT_X3
f,ax = plt.subplots(figsize=(10, 6))
g = sns.heatmap(fulldata[["PACK_VOICE_Q_X1", "PACK_VOICE_Q_X2", "PACK_VOICE_Q_X3", "PACK_VOICE_AMT_X1",
                     "PACK_VOICE_AMT_X2", "PACK_VOICE_AMT_X3"]].corr(),
            annot=True, linewidths=.5, fmt= '.1f', ax=ax)
plt.suptitle("Montos Pack voz mensual y Cantidad de pack voz mensual \nCorrelation Heatmap",
               fontsize=16, weight="bold", y=1)


In [ ]:
# Vemos que tienen una alta correlacion elimino las los montos
fulldata.drop(columns=['PACK_VOICE_AMT_X1','PACK_VOICE_AMT_X1','PACK_VOICE_AMT_X1'], inplace=True)

Comparacion entre Montos pack datos mensual y Cantidad de pack datos mensual  PACK_DATA_Q_X1 PACK_DATA_Q_X2 PACK_DATA_Q_X3  PACK_VOICE_AMT_X1  PACK_VOICE_AMT_X2  PACK_VOICE_AMT_X3 --ANDRE







In [ ]:
#Comparacion entre Montos pack voz mensual y Cantidad de pack voz mensual  PACK_DATA_Q_X1 PACK_DATA_Q_X2 PACK_DATA_Q_X3 PACK_DATA_AMT_X1 PACK_DATA_AMT_X2 PACK_DATA_AMT_X3

f,ax = plt.subplots(figsize=(10, 6))
g = sns.heatmap(fulldata[["PACK_DATA_Q_X1", "PACK_DATA_Q_X2", "PACK_DATA_Q_X3", "PACK_DATA_AMT_X1",
                     "PACK_DATA_AMT_X2", "PACK_DATA_AMT_X3"]].corr(),
            annot=True, linewidths=.5, fmt= '.1f', ax=ax)
plt.suptitle("Montos Pack datos mensual y Cantidad de pack datos mensual \nCorrelation Heatmap",
               fontsize=16, weight="bold", y=1)


In [ ]:
# Vemos que tienen una alta correlacion elimino las los montos
fulldata.drop(columns=['PACK_DATA_AMT_X1','PACK_DATA_AMT_X2','PACK_DATA_AMT_X3'], inplace=True)


Comparacion entre Montos pack datos mensual y Cantidad de pack datos mensual  PACK_SMS_Q_X1  PACK_SMS_Q_X2  PACK_SMS_Q_X3  PACK_SMS_AMT_X1  PACK_SMS_AMT_X2  PACK_SMS_AMT_X3 --ANDRE







In [ ]:
#Comparacion entre Montos pack mensual y Cantidad de pack SMS  PACK_SMS_Q_X1 PACK_SMS_Q_X2 PACK_SMS_Q_X3 PACK_SMS_AMT_X1 PACK_SMS_AMT_X2 PACK_SMS_AMT_X3

f,ax = plt.subplots(figsize=(10, 6))
g = sns.heatmap(fulldata[["PACK_SMS_Q_X1", "PACK_SMS_Q_X1", "PACK_SMS_Q_X1", "PACK_SMS_AMT_X1",
                     "PACK_SMS_AMT_X2", "PACK_SMS_AMT_X3"]].corr(),
            annot=True, linewidths=.5, fmt= '.1f', ax=ax)
plt.suptitle("Montos Pack SMS mensual y Cantidad de pack SMS mensual \nCorrelation Heatmap",
               fontsize=16, weight="bold", y=1)


In [ ]:
# Vemos que tienen una alta correlacion elimino las los montos--ANDREUPDATED
#fulldata.drop(columns=['PACK_DATA_AMT_X1','PACK_DATA_AMT_X2','PACK_DATA_AMT_X3'], inplace=True)
#no tiene valores borro todo 
plt.figure(figsize=(15,7));

plt.hist(fulldata["PACK_SMS_Q_X1"], bins=40, alpha=0.5, label='Q SMS X1', color = "black") 
plt.hist(fulldata["PACK_SMS_Q_X2"], bins=40, alpha=0.5, label='Q SMS X2', color = "brown") 
plt.hist(fulldata["PACK_SMS_Q_X3"], bins=40, alpha=0.5, label='Q SMS X2', color = "red") ;

plt.title('Cantidad mensuales de SMS');

plt.ylabel('Frecuencia')
plt.xlabel('Cantidad de recargas SMS');
plt.legend(loc="center right")
plt.grid(color='grey', linestyle='solid');






In [ ]:
#-ANDRENEW
plt.figure(figsize=(15,7));

plt.hist(fulldata["PACK_SMS_AMT_X1"], bins=40, alpha=0.5, label='Monto SMS X1', color = "green") 
plt.hist(fulldata["PACK_SMS_AMT_X1"], bins=40, alpha=0.5, label='Monto SMS X2', color = "grey") 
plt.hist(fulldata["PACK_SMS_AMT_X1"], bins=40, alpha=0.5, label='Monto SMS X3', color = "pink");

plt.title('Monto mensuales de SMS');
plt.ylabel('Frecuencia')
plt.xlabel('Monto de recargas SMS');
plt.legend(loc="center right")
plt.grid(color='grey', linestyle='solid');

In [ ]:
# No tiene valores borro esas columnas --ANDRE
fulldata.drop(columns=['PACK_SMS_Q_X1','PACK_SMS_Q_X2','PACK_SMS_Q_X3','PACK_SMS_AMT_X1','PACK_SMS_AMT_X2','PACK_SMS_AMT_X3'], inplace=True)


In [ ]:
 #fulldata['TRS_ONNET_SMS_M1'].describe()
(fulldata[[x for x in fulldata.columns if 'SMS' in x]]==0).sum()/fulldata.shape[0]

Analisis datos SMS 

In [ ]:
#Trafico de SMS en la misma red--ANDREUPDATED
plt.figure(figsize=(15,7))
plt.hist(fulldata['TRS_ONNET_SMS_M1'], bins=80, alpha=0.5, label='Trafico SMS M1', color = "purple");
plt.hist(fulldata['TRS_ONNET_SMS_M2'], bins=80, alpha=0.5, label='Trafico SMS M2', color = "green");
plt.hist(fulldata['TRS_ONNET_SMS_M3'], bins=80, alpha=0.5, label='Trafico SMS M3', color = "blue");
plt.title('Trafico de mensajes en la misma red');
plt.ylabel('Frecuencia')
plt.xlabel('Trafico SMS misma red');
plt.legend(loc="center right")
plt.grid(color='grey', linestyle='solid');

In [ ]:
#Como son muy cercanos a 0 - borro estos campos TRS_ONNET_SMS_M1 TRS_ONNET_SMS_M2 TRS_ONNET_SMS_M3
fulldata.drop(columns=['TRS_ONNET_SMS_M1','TRS_ONNET_SMS_M2','TRS_ONNET_SMS_M3'], inplace=True)

In [ ]:
#Trafico de mensajes en otras redes --ANDREUPDATED
plt.figure(figsize=(15,7))
plt.hist(fulldata['TRS_OFFNET_SMS_M1'], bins=80, alpha=0.5, label='Trafico SMS OFF M1', color = "yellow");
plt.hist(fulldata['TRS_OFFNET_SMS_M2'], bins=80, alpha=0.5, label='Trafico SMS OFF M1', color = "green");
plt.hist(fulldata['TRS_OFFNET_SMS_M3'], bins=80, alpha=0.5, label='Trafico SMS OFF M1', color = "blue");
plt.title('Trafico de mensajes en otras redes');
plt.ylabel('Frecuencia')
plt.xlabel('Trafico SMS otras redes');
plt.legend(loc="center right")
plt.grid(color='grey', linestyle='solid');

In [ ]:
#Como son muy cercanos a 0 - borro estos campos TRS_OFFNET_SMS_M1 TRS_OFFNET_SMS_M2 TRS_OFFNET_SMS_M3
fulldata.drop(columns=['TRS_OFFNET_SMS_M1','TRS_OFFNET_SMS_M2','TRS_OFFNET_SMS_M3'], inplace=True)

In [ ]:
fulldata[['TRD_M1','TRD_STR_M1','TRD_SN_M1','TRD_IM_M1']].head()

In [ ]:
fulldata[['TRD_STR_M1','TRD_SN_M1','TRD_IM_M1']].div(1024).sum(axis=1)

In [ ]:
corr = fulldata.corr()

In [ ]:
plt.figure(figsize=(60,60))
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)


In [ ]:
plt.figure(figsize=(60,60))
# plot the heatmap
sns.heatmap(corr, 
        annot = True, vmin=-1, vmax=1, center= 0,
        xticklabels=corr.columns,
        yticklabels=corr.columns)

In [ ]:
kot = corr[corr>=.6]
plt.figure(figsize=(60,60))
sns_plot = sns.heatmap(kot, cmap="Greens")
sns_plot.figure.savefig("output.png")

Train test
Debemos separar el modelo en 2 partes:
    * Conjunto entrenamiento 
    * Conjunto de test
Entonces a partir del conjunto de entrenamiento, entrenamos el modelo, y una vez que conseguimos un modelo predictivo, con el conjunto de test testeamos el modelo para ver que tan bien funciona

Necesitamos hacer una division de lo que se considera datos de entrenamiento (data) y el resultado a obtener (tag)

In [ ]:
fulldata.sample()

In [ ]:
fulldata.shape

In [ ]:
#Nuestro tag estara dado por la columna target, al que llamaremos y 
y = fulldata['Target']

Ahora tenemos que crear la variable x que contendra la informacion para el entrenamiento del modelo (data)

Tenemos que quitar la columna SUSCRIBER_KEY que es una columna de valores unicos y que solo sirve para identificar a cada linea, pero no para entrenar el modelo. Y obviamente tampoco tiene sentido poner la columna Target dentro de los datos de 
entrenamiento, sino el modelo va a conocer perfectamente que persona abandona o no el servicio

Y solamente tomamos en cuenta las columnas que son numericas

In [ ]:
fulldata = fulldata.select_dtypes(include=['int64','float64'])

In [ ]:
columns = [col for col in fulldata.columns if col not in [ 'SUSCRIBER_KEY', 'Target']]

In [ ]:
X = fulldata[columns]
X.sample(5)

In [ ]:
#ahora nos quedamos con los valores de cada variable
#values pasa de serie a array
y = y.values
X = X.values

Ahora que tenemos las dos variables X e y, debemos realizar una division del dataset en un dataset de entrenamiento y uno de testeo

Usaremos el metodo Train Test Split, el cual se encarga de dividir nuestro conjunto original en un conjunto de entrenamiento y un conjunto de testeo

In [ ]:
from sklearn.model_selection import train_test_split # cargamos el metodo

In [ ]:
#Separamos el conjunto entrenamiento y test con el metodo train_test_split y se lo asignamos a las distintas variables
X_train, X_test, y_train, y_test = train_test_split(X,y)
#por default el conjunto se separara en 75% para el train y 25% para el test

In [ ]:
X.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Podemos observar que como separamos la variable X en train y test, la suma de registros de X_train y X_test 
dara la cantidad de registros de X. En el caso de la variable y ocurriria exactamente igual.

A su vez, X_train tendra la misma cantidad de registros o filas que y_train, y X_test la misma que y_test.

In [ ]:
#utilizaremos xgboost como modelo predictivo
!pip install xgboost

In [ ]:
#### Imports para entrenamiento, predicción y evaluación del modelo ####
#### NO QUITAR ####
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
#estas herramientas nos permiten calcular metricas del modelo 

In [ ]:
def train(X_train,y_train):
    """
    Función para entrenar el modelo.
    
    Parameters
    -----------
    
    X_train: pd.DataFrame
        Dataset de Train, listo para entrenar y sin tag
        
    y_train: pd.DataFrame
        Dataset de tag, es el tag correspondiente al dataset de train.
        
    Return
    ----------
    Modelo Entrenado.
    
    """
    xgb_model = xgb.XGBClassifier()
    #instanciamos el clasificador xgb
    
    print("Comienza entrenamiento del modelo XGBoost")

    xgb_model.fit(X_train, y_train)
    #el metodo fit llama al modelo matematico que entrenara el modelo 
    #xgb model ahora va a ser un modelo entrenado
    
    print("Entrenamiento finalizado\n")

    print("Haciendo las predicciones\n\n")

    #con este modelo ya entrenado llamo al metodo predict
    XGB_preds = xgb_model.predict(X_test)
    #comparo mis predicciones con el valor real que tengo guardo en y_test 
    print(classification_report(y_test, XGB_preds))
    print(confusion_matrix(y_test, XGB_preds))
    return xgb_model

In [ ]:
def predict(data_to_predict, xgb_model):
    """Funcion para hacer el entrenamiento del modelo y guardarlo en un archivo .pkl

    Parameters
    -----------

    data_to_predict : pd.DataFrame
        DataFrame listo para predecir, con las mismas transformaciones que el de entrenamiento.
        
    xgb_model : xgb.XGBClassifier
        Modelo entrenado de XGBoost

    Return
    -----------
    Dataframe de 3 columnas, la identificacion del cliente, la prediccion de la clase y la probabilidad de que el cliente
    permanezca en el servicio.


    """
    print("\n\n-------------------------")
    print("Ingreso a la funcion de prediccion\n")

    XGB_preds = xgb_model.predict_proba(X)
    #si en vez de un predict proba hacemos un predict nos daria como resultado 0 y 1 directamente, redondeando
    
    return XGB_preds

In [ ]:
modelo = train(X_train, y_train)

Obtenemos la prediccion del modelo La metrica que mas interesa ver es f1-score .. 
vemos qeue el modelo predice mejor 0s que 1s, tiene sentido porque tenemos mas 0s que 1s entre los datos

In [ ]:
resultado = predict(X_test,modelo)

In [ ]:
resultado
#podemos ver un array con las probabilidades de que cada registro valga 0 o 1
#por ejemplo en el primer registro tenemos 86% de probabilidades de que valga 0, y 13% de que valga 1

In [ ]:
resultado = pd.DataFrame(resultado)

In [ ]:
#podriamos determinar por ejemplo que tengan 1 aquellos que tengan un valor de probabilidad mayor a 0.6 
(resultado[1] > 0.6).astype(int)

In [ ]:
pd.DataFrame(resultado).to_csv('resultado.csv')